<a href="https://colab.research.google.com/github/NickStrauch13/beverage-recommender/blob/tom/beer_ncf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()
print("files done")

Saving kaggle.json to kaggle.json
files done


In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets list

ref                                                           title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                           Online Food Dataset                               3KB  2024-03-02 18:50:30          24897        495  0.9411765        
sukhmandeepsinghbrar/most-subscribed-youtube-channel          Most Subscribed YouTube Channel                   1KB  2024-04-10 20:33:05            792         30  1.0              
sanyamgoyal401/customer-purchases-behaviour-dataset           Customer Purchases Behaviour Dataset              1MB  2024-04-06 18:42:01           1328         34  1.0              
mohdshahnawazaadil/credit-card-dataset                        Credit Card Dataset         

In [6]:
!kaggle datasets download -d rdoume/beerreviews

 91% 25.0M/27.4M [00:01<00:00, 28.8MB/s]
100% 27.4M/27.4M [00:01<00:00, 21.6MB/s]


In [7]:
!unzip beerreviews.zip

Archive:  beerreviews.zip
  inflating: beer_reviews.csv        


In [8]:
import pandas as pd
df = pd.read_csv('beer_reviews.csv')

In [9]:
df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


In [10]:
!pip install tensorflow pandas numpy scikit-learn

In [11]:
ndf = df[['review_profilename', 'beer_beerid', 'review_overall']]

In [12]:
print("Missing values before handling:")
print(ndf.isnull().sum())

ndf = ndf.dropna()
print("Missing values after handling:")
print(ndf.isnull().sum())

print(ndf.head())

Missing values before handling:
review_profilename    348
beer_beerid             0
review_overall          0
dtype: int64
Missing values after handling:
review_profilename    0
beer_beerid           0
review_overall        0
dtype: int64
  review_profilename  beer_beerid  review_overall
0            stcules        47986             1.5
1            stcules        48213             3.0
2            stcules        48215             3.0
3            stcules        47969             3.0
4     johnmichaelsen        64883             4.0


In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

In [14]:
# Create a dictionary mapping beer IDs to names
beer_id_to_name = pd.Series(df.beer_name.values, index=df.beer_beerid).to_dict()

# Encode 'review_profilename' and 'beer_beerid'
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ndf['user_encoded'] = user_encoder.fit_transform(ndf['review_profilename'])
ndf['item_encoded'] = item_encoder.fit_transform(ndf['beer_beerid'])

# Split the dataset
train, test = train_test_split(ndf, test_size=0.2, random_state=42)

In [15]:
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

def create_ncf(num_users, num_items, embedding_size, layer_sizes=[64, 32, 16, 8]):
    # Input layers
    user_input = layers.Input(shape=(1,))
    item_input = layers.Input(shape=(1,))

    # Embeddings layers
    user_embedding_gmf = layers.Embedding(num_users, embedding_size, embeddings_initializer='he_normal',
                                          embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)
    user_embedding_gmf = layers.Flatten()(user_embedding_gmf)

    item_embedding_gmf = layers.Embedding(num_items, embedding_size, embeddings_initializer='he_normal',
                                          embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(item_input)
    item_embedding_gmf = layers.Flatten()(item_embedding_gmf)

    user_embedding_mlp = layers.Embedding(num_users, embedding_size, embeddings_initializer='he_normal',
                                          embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)
    user_embedding_mlp = layers.Flatten()(user_embedding_mlp)

    item_embedding_mlp = layers.Embedding(num_items, embedding_size, embeddings_initializer='he_normal',
                                          embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(item_input)
    item_embedding_mlp = layers.Flatten()(item_embedding_mlp)

    # GMF part
    gmf_mul = layers.Multiply()([user_embedding_gmf, item_embedding_gmf])

    # MLP part
    mlp_concat = layers.Concatenate()([user_embedding_mlp, item_embedding_mlp])
    mlp = mlp_concat
    for layer_size in layer_sizes:
        mlp = layers.Dense(layer_size, activation='relu')(mlp)

    # Concatenate GMF and MLP parts
    concat = layers.Concatenate()([gmf_mul, mlp])

    # Final prediction layer scaled to 0-5 range
    output = layers.Dense(1, activation='linear')(concat)
    output = layers.Lambda(lambda x: x * 5)(output)

    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])

    return model


In [17]:
# Instantiate and train the model
num_users = ndf['user_encoded'].nunique()
num_items = ndf['item_encoded'].nunique()
embedding_size = 256
model = create_ncf(num_users, num_items, embedding_size)
model.summary()

# Preparing the data
x_train = [train['user_encoded'].values, train['item_encoded'].values]
y_train = train['review_overall'].values

# Training the model
history = model.fit(x_train, y_train, batch_size=128, epochs=4, validation_split=0.2)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, 1, 256)               8547072   ['input_3[0][0]']             
                                                                                                  
 embedding_7 (Embedding)     (None, 1, 256)               1690905   ['input_4[0][0]']             
                                                          6                                 

In [18]:
# Preparing the data
x_test = [test['user_encoded'].values, test['item_encoded'].values]
y_test = test['review_overall'].values  # Use actual ratings

# Evaluate the model using regression metrics
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f'Test MSE: {test_loss}, Test MAE: {test_mae}')

9915/9915 [==============================] - 41s 4ms/step - loss: 0.5208 - mae: 0.5253
Test MSE: 0.5207662582397461, Test MAE: 0.5253141522407532


In [19]:
import numpy as np
import pandas as pd

# Example setup
user_to_predict = 'stcules'
beers_to_predict = [47986, 64883]

# Ensure that user_encoder and item_encoder are already defined and fitted
# Transform the user string to index using LabelEncoder
user_index = user_encoder.transform([user_to_predict])[0]

# Iterate over the list of beer IDs to predict
for beer_id in beers_to_predict:
    # Transform the beer ID to index
    try:
        item_index = item_encoder.transform([beer_id])[0]
        # Make the prediction using the transformed indices
        prediction = model.predict([np.array([user_index]), np.array([item_index])])

        # Fetch the beer name using the ID, default to 'Unknown Beer' if ID is not found
        beer_name = beer_id_to_name.get(beer_id, 'Unknown Beer')

        # Print the prediction
        print(f"Predicted rating for {beer_name} (ID: {beer_id}): {prediction[0][0]:.2f}")
    except ValueError:
        print(f"Beer ID {beer_id} not found in the training data.")


1/1 [==============================] - 0s 124ms/step
Predicted rating for Sausa Weizen (ID: 47986): 1.05
1/1 [==============================] - 0s 17ms/step
Predicted rating for Cauldron DIPA (ID: 64883): 3.18
